# TP4: Regression logistique (pédestre)

En fin de séance, votre TP devra être déposé sur le dépot:
https://synapse.math.univ-toulouse.fr/s/4kEQGz0uay4KkCo

**Attention: Je n'accepte pas les TPs d'élèves absents en cours.**

Nous nous intéressons au dataset public [Framingham](https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset)
issu d'une étude en 1948 à Framingham, Massaschussets, USA et coordonnée par le U.S. Public Health Service. Je vous propose la version [ici](https://www.math.univ-toulouse.fr/~rchhaibi/teaching/2019/M1SID/framingham_fr.csv)
où j'ai simplement traduit les champs. Vous pouvez consultez les kernels disponibles, mais le but de ce TP est une implémentation plus manuelle et pédestre de la régression linéaire.

Description par le gouvernement américain: [Lien](https://biolincc.nhlbi.nih.gov/studies/framcohort/)
- Pour les variables binaires : “1”=“Oui”, “0”=“Non”.
- Pour les variables continues: Valeur intensive

Variable d'intérêt en dernière colonne:
Risque à 10 ans de développer une maladie coronaire (binaire)

* Facteurs démographiques:
  * Genre: Masculin ou Féminin (binaire: "1"=Masculin)
  * Age: Continu
  * Education: Niveau d'éducation 1,2,3,4
* Facteurs comportementaux:
  * Fumeur: (binaire)
  * CigsParJour: Cigarette par jour
* Facteurs médicaux historiques / Historique médical:
  * meds (binaire): si le patient est traité pour des problèmes de pression sanguine
  * avc  (binaire): si le patient a déjà fait un avc
  * hypertension (binaire): si le patient a de l'hypertension
  * diabete (binaire): si le patient est diabétique
* Facteurs médicaux courants:
  * Tot Chol: niveau de cholesterol total HDL + LDL + VLDL (Continu)
  * Sys BP: pression sanguine systolique (Continu)
  * Dia BP: pression sanguine diastolique (Continu)
  * IMC: Indice de Masse Corporelle (Continu)
  * freqCardiaque: Fréquence cardiaque (Continu)
  * Glucose: niveau de glucose (Continu)

In [2]:
## Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## I. Echauffement: L'algorithme de Newton-Raphson

Exercice 1:
- Implémenter l'algorithme de Newton-Raphson dans sa version scalaire
- De même, dans sa version vectorielle
- Commenter la vitesse de convergence

In [3]:
# Version scalaire
def newton_raphson_scalar( x0, func, grad_func, n_max=100, eps=1e-8):
    x = x0
    x0= x + 2*eps
    k = 0
    while abs(x-x0)>eps and k<n_max:
        x0 = x
        x  = x0 - func(x0)/grad_func(x0)
        k  = k + 1
    return (x, k)

In [4]:
# Test pour la version scalaire
a    = 2
scalar_func = lambda x: x**2-a
scalar_grad = lambda x: 2*x

count  = 8
errors = []
print("Target value: ", np.sqrt(a))
print("NR value / Iteration count:")
for n_max in range(1,count):
    x, k = newton_raphson_scalar(1, scalar_func, scalar_grad, n_max=n_max)
    print(x, k)

Target value:  1.4142135623730951
NR value / Iteration count:
1.5 1
1.4166666666666667 2
1.4142156862745099 3
1.4142135623746899 4
1.4142135623730951 5
1.4142135623730951 5
1.4142135623730951 5


## II. Exploration de données

Exercice 2:
- Lire le code suivant.
- Pourquoi elève-t-on la colonne "éducation"? Il s'agit pourant (sans doute) d'un facteur pertinent.

,masculin,age,fumeur,cigsParJour,meds,avc,hypertension,diabete,totChol,sysBP,diaBP,IMC,freqCardiaque,glucose,risque10ans
0,1,39,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [50]:
#Importation des données et nettoyage
dataframe=pd.read_csv("framingham_fr.csv")
dataframe.drop(['education'],axis=1,inplace=True)
dataframe.head()
#Counting the missing values and dropping them
count=0
for i in dataframe.isnull().sum(axis=1):
    if i>0:
        count=count+1
print("Le nombre total de lignes avec des valeurs manquantes est ", count)
print("Il s'agit de",round((count/len(dataframe.index))*100), '% du jeu de données.')
print("")
dataframe.dropna(axis=0,inplace=True)

#Ajout de la constante
from statsmodels.tools import add_constant as add_constant
dataframe = add_constant(dataframe)

# Statistiques descriptives
print("Statistiques descriptives:")
dataframe.describe()

Le nombre total de lignes avec des valeurs manquantes est  489
Il s'agit de 12 % du jeu de données.

Statistiques descriptives:


,const,masculin,age,fumeur,cigsParJour,meds,avc,hypertension,diabete,totChol,sysBP,diaBP,IMC,freqCardiaque,glucose,risque10ans
count,3749.0,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000
mean,1.0,0.445185,49.578821,0.488397,9.005335,0.030408,0.005601,0.311816,0.027207,236.952787,132.365964,82.933716,25.809651,75.703921,81.883169,0.152574
std,0.0,0.497053,8.569322,0.499932,11.922440,0.171730,0.074643,0.463297,0.162709,44.610417,22.051951,11.933321,4.065894,11.957763,23.888039,0.359624
min,1.0,0.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000,83.500000,48.000000,15.540000,44.000000,40.000000,0.000000
25%,1.0,0.000000,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,117.000000,75.000000,23.090000,68.000000,71.000000,0.000000
50%,1.0,0.000000,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,128.000000,82.000000,25.410000,75.000000,78.000000,0.000000
75%,1.0,1.000000,56.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,264.000000,144.000000,90.000000,28.060000,82.000000,87.000000,0.000000
max,1.0,1.000000,70.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.000000,142.500000,56.800000,143.000000,394.000000,1.000000


In [54]:
print(dataframe.isnull().any())

const            False
masculin         False
age              False
fumeur           False
cigsParJour      False
meds             False
avc              False
hypertension     False
diabete          False
totChol          False
sysBP            False
diaBP            False
IMC              False
freqCardiaque    False
glucose          False
risque10ans      False
dtype: bool


In [55]:
cols = dataframe.columns
# Définition de la variable d'intéret
Y = np.array( dataframe[cols[-1]] )
# Définition de la matrice des facteurs
X = np.array( dataframe[cols[:-1]] )
# Taille des matrices
print("dim X: ", X.shape)
print("dim Y: ", Y.shape)


dim X:  (3749, 15)
dim Y:  (3749,)


## III. Implémentation de la régression logistique

Nous nous intéressons à l'estimation de 
$$ \mathbb{P}\left( Y=1 | X \right)$$
grâce à un modèle de régression logistique. On pourra se référer à la [page Wikipedia](https://fr.wikipedia.org/wiki/R%C3%A9gression_logistique#Le_mod%C3%A8le).

- La fonction de lien est définie pour $p = \mathbb{P}\left( Y = 1 | X \right) = \mathbb{E}\left( Y | X \right)$ par
$$ 
   \textrm{logit}( p )
 = \log\left( \frac{p}{1-p} \right)
 = \beta X \ ,
$$
où $\beta X$ est le produit scalaire entre les paramètres $\beta \in \mathbb{R}^k$ et les facteurs $X \in \mathbb{R}^k$. De façon équivalente:
$$ p = \frac{e^{\beta X}}{1+e^{\beta X}} \ .$$

- Dans ce cas, la log-vraisemblance s'écrit
$$
   \ell\left( y, x, \beta \right)
 = \frac{1}{n} \sum_{i=1}^n \left[
   y_i \left( \beta x_i \right)
   - \log (1+e^{\beta x_i})
   \right]
$$
Preuve:
$$
   \begin{align*}
   \ell\left( y, x, \beta \right) := & \frac{1}{n} \log \mathbb{P}\left( \forall i, \ Y_i = y_i | \forall i, \ X_i = x_i \right) \\
   = & \frac{1}{n} \sum_{i=1}^n \log \mathbb{P}\left( Y_i = y_i | X_i = x_i \right) \\
   = & \frac{1}{n} \sum_{i=1}^n 
   \left[
   y_i \log \mathbb{P}\left( Y_i = 1 | X_i = x_i \right)
   + (1-y_i) \log \mathbb{P}\left( Y_i = 0 | X_i = x_i \right)
   \right]
   \\
   = & \frac{1}{n} \sum_{i=1}^n \left[
   y_i \log \frac{e^{\beta x_i}}{1+e^{\beta x_i}}
   + (1-y_i) \log \frac{1}{1+e^{\beta x_i}}
   \right]
   \\
   = & \frac{1}{n} \sum_{i=1}^n \left[
   y_i \left( \beta x_i \right)
   + \log \frac{1}{1+e^{\beta x_i}}
   \right]
   \\
   = & \frac{1}{n} \sum_{i=1}^n \left[
   y_i \left( \beta x_i \right)
   - \log (1+e^{\beta x_i})
   \right] \ .
   \\
   & \qquad \qquad \qquad \qquad \qquad \qquad CQFD
   \end{align*}
$$
- En conséquence:
$$
   \nabla_\beta \ell\left( y, x, \beta \right)
 = \frac{1}{n} \sum_{i=1}^n \left(
   y_i
   -
   \frac{e^{\beta x_i}}{1+e^{\beta x_i}}
   \right) x_i
 = \frac{1}{n} \sum_{i=1}^n \left(
   y_i
   - 
   \frac{1}{1+e^{-\beta x_i}}
   \right) x_i
 = \frac{1}{n} V X \ ,
$$
$$
   \nabla_\beta^2 \ell\left( y, x, \beta \right)
 = \frac{1}{n} \sum_{i=1}^n 
   \frac{-e^{-\beta x_i}}{(1+e^{-\beta x_i})^2}
   x_i^T x_i 
 = \frac{1}{n} \sum_{i=1}^n 
   \frac{-1}{2 + e^{-\beta x_i} + e^{\beta x_i}}
   x_i^T x_i 
 = \ \frac{1}{n} X^T W X,
$$
où $V$ et $W$ sont des matrices explicites


Exercice 3:
Refaire les calculs ci-dessus et écrire trois fonctions
- ${\it logL}$: Calcul de la log vraisemblance (Fait)
- ${\it diff\_logL}$ : Calcul de son gradient
- ${\it diff2\_logL}$: Calcul de la matrice Hessienne


In [63]:
def logL( beta, y, x):
    result  = 0
    betaX   = np.dot(x, beta)
    y_betaX = y*betaX
    result  = np.sum( y_betaX-np.log(1+np.exp(betaX)) )
    result  = result/len(y)
    return result


def diff_logL( beta, y, x):
    result  = 0
    betaX   = np.dot(x, beta)
    V       = y-1/(1+np.exp(-betaX))
    result  = np.dot(V, x)/len(y)
    return result

def diff2_logL( beta, y, x):
    result = 0
    betaX  = np.dot(x, beta)
    W      = -1/( 2 + np.exp(-betaX) + np.exp( betaX) )
    W      = np.diag(W)
    result = np.dot( x.transpose(), np.dot(W, x) )
    result = result/len(y)
    return result


In [64]:
# Test
k   = X.shape[1]
np.random.seed(1)
beta0 = np.random.rand(k)/100
print("Log-vraisemblance initiale: ", logL(beta0, Y, X))
print("Gradient  initial : ", diff_logL(beta0, Y, X))
print("Hessienne initiale: ", diff2_logL(beta0, Y, X).shape )


Log-vraisemblance initiale:  -2.702928648890207
Gradient  initial :  [-8.04672721e-01 -3.40834526e-01 -3.92211519e+01 -3.88805784e-01
 -7.00930908e+00 -1.96074684e-02 -3.26075889e-03 -2.24113974e-01
 -1.66543101e-02 -1.89711926e+02 -1.04955098e+02 -6.61937829e+01
 -2.06622787e+01 -6.08930786e+01 -6.48631477e+01]
Hessienne initiale:  (15, 15)


Exercice 4:
- Ecrire la version vectorielle de Newton-Raphson donnée en cours
- Fitter le modèle en maximisant la log vraisemblance par l'algorithme de Newton-Raphson

In [66]:
# Version vectorielle de Newton-Raphson
def newton_raphson( x0, func, jacobian_func, n_max=100, eps=1e-8, optional=None):
    x = x0
    x0= x + 2*eps
    k = 0
    while np.max(np.abs(x-x0))>eps and k<n_max:
        x0 = x
        if optional:
            print("Step: ", k)
            print("Likelihood: ", optional(x0))
        jacobian = jacobian_func(x0)
        jacobian_inv = np.linalg.inv(jacobian)
        x  = x0 - np.dot(jacobian_inv, func(x0) )
        k  = k + 1
    return (x, k)

# Fitting
k     = X.shape[1]
beta0 = np.zeros(k)
l     = lambda beta : logL(beta, Y, X)
diff  = lambda beta : diff_logL(beta, Y, X) 
diff2 = lambda beta : diff2_logL(beta, Y, X) 
result = newton_raphson( beta0, diff, diff2, optional=l)
print("")

#Final values
print(" Likelihood: %f \n Number of iterations: %d \n Estimated beta:" % (l(result[0]), result[1]), result[0] )

Step:  0
Likelihood:  -0.6931471805599452
Step:  1
Likelihood:  -0.39984836887355196
Step:  2
Likelihood:  -0.37881637316085437
Step:  3
Likelihood:  -0.3772162644368042
Step:  4
Likelihood:  -0.37719896260846303
Step:  5
Likelihood:  -0.37719896004663
Step:  6
Likelihood:  -0.37719896004662995

 Likelihood: -0.377199 
 Number of iterations: 7 
 Estimated beta: [-8.64629355e+00  5.73994317e-01  6.40449952e-02  7.31683648e-02
  1.83713528e-02  1.44564752e-01  7.19071651e-01  2.14630016e-01
  2.47120514e-03  2.24375667e-03  1.53448289e-02 -3.93363611e-03
  1.02649003e-02 -2.28503447e-03  7.57614086e-03]


## IV. Comparaison à statmodels

Bien entendu, ce que vous venez d'implémenter est standard et est contenu dans les packages python appropriés. 

Comparez vos résultats à:

In [16]:
import statsmodels.api as sm

cols=dataframe.columns[:-1]
model=sm.Logit(dataframe.risque10ans,dataframe[cols])
result=model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.377199
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            risque10ans   No. Observations:                 3749
Model:                          Logit   Df Residuals:                     3734
Method:                           MLE   Df Model:                           14
Date:                Mon, 27 Apr 2020   Pseudo R-squ.:                  0.1169
Time:                        13:53:24   Log-Likelihood:                -1414.1
converged:                       True   LL-Null:                       -1601.4
                                        LLR p-value:                 2.922e-71
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -8.6463      0.687    -12.577      0.000      -9.994      -7.299
masculin          0.5740      0.107      5.343      0.000       0.363       0.785
age               0.0640      0.007      9.787      0.000       0.051       0.077
fumeur            0.0732      0.155      0.473      0.636      -0.230       0.376
cigsParJour       0.0184      0.006      3.003      0.003       0.006       0.030
meds              0.1446      0.232      0.622      0.534      -0.311       0.600
avc               0.7191      0.489      1.471      0.141      -0.239       1.677
hypertension      0.2146      0.136      1.574      0.116      -0.053       0.482
diabete           0.0025      0.312      0.008      0.994      -0.609       0.614
totChol           0.0022      0.001      2.074      0.038       0.000       0.004
sysBP             0.0153      0.004      4.080      0.000       0.008       0.023
diaBP            -0.0039      0.006     -0.619      0.536      -0.016       0.009
IMC               0.0103      0.013      0.820      0.412      -0.014       0.035
freqCardiaque    -0.0023      0.004     -0.550      0.583      -0.010       0.006
glucose           0.0076      0.002      3.408      0.001       0.003       0.012
=================================================================================
"""